# Computer Lab 1: k-NN classifier

## Exercise 3 – User localization from RSSI

Consider the following scenario, in which we wish to localize a user employing a non-GPS system  (e.g., in indoor localization). The user holds a transmission device (e.g., a smartphone or other sensor with transmission capabilities). Localization is based on measurements of the Received Signal Strength Indicator (RSSI) from D sensors (base stations) placed in the area in which the localization service is provided. The area is divided into $N_C$ square cells, and localization amounts to identifying the cell in which the user is located.

In a **training stage**, the transmission device is placed in the center of each cell and broadcasts a data packet, and RSSI is measured by each sensor. This yields one measurement, corresponding to a vector of length $D$. The process is repeated $M$ times for each cell, and for all $N_C$ cells. The training stage provides a 3-dimensional array of size $N_C \times D \times M$.

In a **test stage**, the user is located in an unknown cell. The transmission device broadcasts a data packet, and each sensor measures the RSSI and communicates it to a fusion center. The fusion center treats the received RSSI values as a test vector of length $D$. It applies a k-NN classifier, comparing the test vector with all $M \times N_C$ training vectors available in the training set. For each test vector, the k-NN classifier outputs the probability that each cell contains the user.

**Available data**: you are provided with a file (`localization.mat` in `/data/` folder) containing two variables, called traindata and testdata. These variables have the same size, and are 3-dimensional arrays of size $D=7$, $M=5$, and $N_C = 24$.

The training data can be seen as labelled data where each cell is a class, and you are given M data vectors for each cell. Regarding the test data, a test vector consists of a single measurement; so each measurement has to be used individually and you can perform up to M tests for each cell.
The data correspond to real acquisition experiments performed outdoors nearby Politecnico di Torino, using an STM32L microcontroller with 915 MHz 802.15.4 transceiver.

**Task**: your task is to implement a k-NN classifier in Matlab for the classification task described above, and evaluate its performance.

**Performance evaluation**: The performance is defined in terms of accuracy in the localization task, and it has to be averaged over all cells. Average accuracy is defined as the posterior probability associated to the cell that the user is actually located in.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.io
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm

# Plots setting.
sns.set_context(
    'talk', rc = {
        'font.size': 12.0,
        'axes.labelsize': 10.0,
        'axes.titlesize': 10.0,
        'xtick.labelsize': 10.0,
        'ytick.labelsize': 10.0,
        'legend.fontsize': 10.0,
        'legend.title_fontsize': 12.0,
        'patch.linewidth': 2.0
        }
    )

data_sets = ['Train', 'Test']

In [2]:
# Check current folder.
os.getcwd()

'/'

In [3]:
data_path = "/Users/ernestocolacrai/Documents/GitHub/StatisticalLearning/data/"

try:
    # Attempt to load the MATLAB data file.
    data = scipy.io.loadmat(data_path + f"localization.mat")

    print(
        f"Data ✓\n",
        f"Data Keys: {data.keys()}"
        )
except:
    print(f"Not found data! ({data_path})")

Data ✓
 Data Keys: dict_keys(['__header__', '__version__', '__globals__', 'cell_coordinates', 'testdata', 'traindata'])


In [4]:
def rearrange(dataset, rows, columns, depth):
    arranged = np.zeros([columns * depth, rows + 1])
    count = 0
    for j in range(depth):
        for i in range(columns):
            arranged[i + count, :-1] = dataset[:, i, j].T
            arranged[i + count, -1] = j + 1
        
        count = count + columns
    return arranged

In [5]:
D = 7 # Features number (ROWS)
M = 5 # Measures number for each cell (class) (COLUMNS)
Nc = 24 # Classes number (cells number) (DEPTH)

len(rearrange(data['traindata'], D, M, Nc)), len(rearrange(data['testdata'], D, M, Nc))

(120, 120)

In [7]:
rearrange(data['traindata'], D, M, Nc)

array([[-32., -49., -46., -71., -54., -63., -50.,   1.],
       [-32., -49., -46., -72., -54., -64., -49.,   1.],
       [-32., -48., -46., -71., -54., -63., -50.,   1.],
       [-32., -48., -47., -71., -54., -64., -50.,   1.],
       [-32., -48., -46., -71., -54., -64., -49.,   1.],
       [-31., -49., -40., -56., -54., -59., -52.,   2.],
       [-31., -49., -40., -56., -54., -59., -52.,   2.],
       [-31., -49., -41., -55., -55., -59., -51.,   2.],
       [-30., -49., -40., -55., -55., -59., -51.,   2.],
       [-30., -49., -40., -55., -55., -59., -51.,   2.],
       [-27., -35., -40., -54., -50., -68., -50.,   3.],
       [-27., -35., -41., -54., -50., -70., -50.,   3.],
       [-27., -35., -40., -53., -50., -71., -50.,   3.],
       [-27., -35., -40., -54., -51., -68., -50.,   3.],
       [-28., -35., -41., -53., -51., -68., -50.,   3.],
       [-28., -44., -45., -54., -61., -66., -45.,   4.],
       [-28., -44., -45., -53., -62., -66., -45.,   4.],
       [-27., -48., -47., -50.,

In [12]:
data['traindata'].shape

(7, 5, 24)